In [1]:
!git clone https://github.com/AndreyK2001/BERT-Classification

Cloning into 'BERT-Classification'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 140 (delta 11), reused 33 (delta 8), pack-reused 103
Receiving objects: 100% (140/140), 46.69 MiB | 22.12 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
import os

os.system("""wget "https://getfile.dokpub.com/yandex/get/https://disk.yandex.com/d/7bfAPhxzUXl5vw" -O VvsW_en""")

0

In [3]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertForSequenceClassification
import pandas as pd
import numpy as np

In [4]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
stete_dict = torch.load("./VvsW_en", map_location='cpu')
model.load_state_dict(stete_dict)

device = "cpu"
if torch.cuda.is_available(): device = "cuda"

model = model.to(device)

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',
                                          do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [11]:
def load_data(language='en'):
    df = pd.read_csv(f'/content/BERT-Classification/2 class wiki/wikidatasests/dataset_{language}.csv', sep=';').dropna()
    return df

def make_dataloader(df, n_samples_per_class=50):

    label_dict = {'Easy': 0, 'Normal': 1}

    Vikidia_texts = pd.DataFrame({'text': df['Vikidia'].sample(n=n_samples_per_class, replace=False).values, 'label': 0})
    Wikipedia_texts = pd.DataFrame({'text': df['Wikipedia'].sample(n=n_samples_per_class, replace=False).values, 'label': 1})
    df = pd.concat([Vikidia_texts, Wikipedia_texts], ignore_index=True)

    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',
                                          do_lower_case=True)

    encoded_data_predict = tokenizer.batch_encode_plus(
        df.text.values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )

    input_ids_predict = encoded_data_predict['input_ids']
    attention_masks_predict = encoded_data_predict['attention_mask']
    labels_predict = torch.tensor(df.label.values)

    dataset_predict = TensorDataset(input_ids_predict, attention_masks_predict, labels_predict)


    batch_size = 3

    return DataLoader(dataset_predict,
                                  sampler=SequentialSampler(dataset_predict),
                                  batch_size=batch_size)

In [12]:
def get_neurons_activations(dataloader):
    model.eval()

    # Подготовка списка для хранения усредненных скрытых состояний по слоям
    avg_hidens_per_layer = [[] for _ in range(13)]  # предполагается, что у модели 12 слоев + входной слой

    for batch in dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        # Создание маски для исключения [PAD] токенов (где input_ids равно 0)
        mask = inputs['input_ids'] != 0  # Создается маска размером как input_ids, True где не PAD
        mask = mask.unsqueeze(-1).expand_as(outputs['hidden_states'][0])  # Расширение маски до размера скрытых состояний

        for layer_idx, layer_hidden_states in enumerate(outputs['hidden_states']):
            # Применение маски к скрытым состояниям
            masked_hidden_states = layer_hidden_states * mask.float()
            # Вычисление суммы и количества не-pad токенов для усреднения
            sum_hidden_states = masked_hidden_states.sum(dim=1)  # Сумма по оси токенов
            non_pad_tokens = mask.sum(dim=1)  # Количество не-pad токенов
            # Усреднение скрытых состояний, исключая pad-токены
            avg_hidden_states = sum_hidden_states / non_pad_tokens.clamp(min=1)  # Избегание деления на 0
            avg_hidens_per_layer[layer_idx].append(avg_hidden_states)

    # Собираем усредненные скрытые состояния по всему датасету для каждого слоя
    avg_hidens_stacked_per_layer = [torch.cat(layer_avg_hidens, dim=0) for layer_avg_hidens in avg_hidens_per_layer]

    # Стекинг усредненных скрытых состояний для всех слоев
    all_avg_hidens_tensor = torch.stack(avg_hidens_stacked_per_layer)

    return all_avg_hidens_tensor # [layers, batch, neurons]


In [13]:
def pearson_corr(x, y):
    return (torch.dot(x, y)/(torch.norm(x)*torch.norm(y))).item()

def ANC(L1, L2):
    X = L1 - L1.mean(dim=0, keepdim=True)
    Y = L2 - L2.mean(dim=0, keepdim=True)

    anc = 0
    for i in range(X.size()[0]):
        anc+=abs(pearson_corr(X[i], Y[i]))

    return anc/X.size()[0]

def count_ANC(layers1, layers2):
    anc = []
    for i in range(13):
        anc.append(ANC(layers1[i], layers2[i]))

    return anc

In [14]:
dataloader_en = make_dataloader(load_data('en'))
layers_en = get_neurons_activations(dataloader_en)
torch.cuda.empty_cache()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# import matplotlib.pyplot as plt

# x = np.arange(0, 13)

# plt.figure(figsize=(10, 6))

# languages = ['es', 'it', 'ru', 'fr']

# for lang in languages:
#     dataloader_compare = make_dataloader(load_data(lang))
#     layers_compare = get_neurons_activations(dataloader_compare)
#     torch.cuda.empty_cache()

#     ANC_counted = count_ANC(layers_en, layers_compare)

#     plt.plot(x, ANC_counted, label=f'en-{lang}', marker='o')

# plt.title('ANC')
# plt.xlabel('Layer')
# plt.ylabel('Value')
# plt.legend()

# plt.grid(True)
# plt.show()

In [15]:
data = []
languages = ['es', 'it', 'ru', 'fr']

for lang in tqdm(languages):
    dataset_compare = load_data(lang)
    for i in range(10):
        dataloader_compare = make_dataloader(dataset_compare)
        layers_compare = get_neurons_activations(dataloader_compare)
        torch.cuda.empty_cache()

        ANC_counted = count_ANC(layers_en, layers_compare)

        data.append({
            'Value': ANC_counted,
            'Language Pair': f'en-{lang}'
        })

df = pd.DataFrame(data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `

In [16]:
df.head()

,Value,Language Pair
0,"[0.162988130080048, 0.13120410955976694, 0.160...",en-es
1,"[0.1497700005210936, 0.12387088591291104, 0.15...",en-es
2,"[0.1746108905831352, 0.1429073861753568, 0.167...",en-es
3,"[0.14736773792421445, 0.1249954819965933, 0.14...",en-es
4,"[0.17198370850645006, 0.1360608158283867, 0.16...",en-es


In [17]:
df.to_csv(f'VvsW.txt', index=False, sep=';')